In [2]:
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag

In [3]:
from tqdm import tqdm
from time import sleep

In [4]:
ress = []

In [5]:
res = requests.get('https://www.della.kz/search/a98bdeflolh0i230301l230516k0m1.html')

In [6]:
res

<Response [200]>

In [7]:
print(str(res.content.decode()))

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html>
<head>

<script type="application/ecmascript" async src="https://www.googletagmanager.com/gtag/js?id=G-F3C76P308L"></script>
<script type="application/ecmascript">
window.dataLayer = window.dataLayer || [];
function gtag() {
dataLayer.push(arguments);
}
gtag('js', new Date());
gtag('config', 'G-F3C76P308L',
{
'user_id': '',
});
</script>

<link rel="stylesheet" href="https://sdk.getrevin.com/v1.css" />
<script async src="https://sdk.getrevin.com/v1.js" data-revin-seller-account-id="62731cc4a7336b2bea31611a"></script>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<title>DELLA&trade; Грузы и Транспорт (прямые заказчики, поиск грузов, попутные грузы, найти груз, свободный и попутный транспорт)</title>
<meta http-equiv="Content-Language" content="ru, kk, av, os, ug">
<meta name="robots" content="NOODP">
<meta name="Des

In [72]:
res = requests.get('https://www.della.kz/search/a98bd98eflolz1z2z3z4z5z6z7z8z9y1y2y3y4y5y6h0ilk0m1.html')

In [73]:
soup = BeautifulSoup(res.text, 'html.parser')

In [74]:
card = soup.find_all(class_='request_card')[0]
card

<div class="request_card request_23071073730611425" data-request_id="23071073730611425">
<div class="request_card_header">
<div class="relevance_request_block">
<img class="time_icon" src="/img/time_icon_minutes.svg"/>
<div class="time_string diff_seconds">
<span class="bold">3 секунды</span> назад</div>
</div>
<div class="request_card_header_right">
<div class="veriff_block request_header_info_block">
<img class="veriff_block_icon" src="/img/veriff_passed.svg"/>
<div class="tooltip_veriff">
<div class="tooltip_body_veriff">Верификация</div>
<div>
<div class="tooltip_arrow_on_bottom"></div>
</div>
</div>
</div>
<div class="country_block request_header_info_block">
<img class="country_flag" src="/img/country_flags/svg/kz.svg"/>
<div class="country_abbr">kz</div>
</div>
<div class="marked_icon not_active without_margin_left check_23071073730611425" data-request_id="23071073730611425" onclick="addRequestToFavorites('request_23071073730611425', true, this,'check_23071073730611425');">
<div

In [75]:
card.select('.request_route span')[::2][:2]

[<span title="Алматинская обл.">
 <span class="locality">Алматы </span>(KZ)</span>,
 <span title="Алматинская обл.">
 <span class="locality">Калжат </span>(KZ)</span>]

In [11]:
from datetime import datetime, timedelta
from dataclasses import dataclass
from urllib.parse import urljoin


@dataclass
class RoutePoint:
    city: str
    country: str
    region: str

@dataclass
class SearchCard:
    creation_time: datetime | None
    country: str | None
    from_location: RoutePoint
    to_location: RoutePoint
    distance: str
    truck_type: str
    weight: str | None
    volume: str | None
    cargo_type: str | None
    date: str
    link: str | None

    def __repr__(self):
        if self.creation_time:
            created_time_ago = (datetime.now() - self.creation_time).total_seconds()
            if created_time_ago < 60:
                created_ago = f'{created_time_ago:.0f} seconds ago'
            elif created_time_ago < 3600:
                created_ago = f'{created_time_ago // 60:.0f} minutes ago'
            else:
                created_ago = f'{created_time_ago // 3600:.0f} hours ago'
        else:
            created_ago = "today"
        
        return (
            f'(Creation time: {created_ago}\n'
            f'From: {self.from_location.city}, {self.from_location.country}, {self.from_location.region}\n'
            f'To: {self.to_location.city}, {self.to_location.country}, {self.to_location.region}\n'
            f'Distance: {self.distance}\n'
            f'Date: {self.date}\n'
            f'Truck type: {self.truck_type}\t'
            f'Weight: {self.weight}\t'
            f'Volume: {self.volume}\t'
            f'Cargo type: {self.cargo_type}\n'
            f'Link: {self.link})\n'
        )




def parse_route_location(location: Tag) -> RoutePoint:
    """Extracts city, country and region from the card route location (request_route)
    
    Args:
        location (Tag): route location tag (span)

    Returns:
        dict: city, country and region
    """
    city_and_country, region = location.text.strip(), location['title']
    city = city_and_country.split(' ')[0]
    country = city_and_country.split(' ')[1][1:-1]
    return RoutePoint(city, country, region)

def extract_info(card_tag: Tag) -> SearchCard:
    """Extracts information from the card tag

    Args:
        card_tag (Tag): card tag

    Returns:
        SearchCard: extracted information
    """

    base_url = 'https://www.della.kz/'

    def extract_text(tag: Tag, selector: str) -> str | None:
        if selected_tag := tag.select_one(selector):
            return selected_tag.text.strip()
        return None



    # Extract time of creation
    if time_string := extract_text(card_tag, '.diff_minutes'):
        minutes = int(time_string.split()[0])
        time_of_creation = datetime.now() - timedelta(minutes=minutes)
    elif time_string := extract_text(card_tag, '.diff_seconds'):
        seconds = int(time_string.split()[0])
        time_of_creation = datetime.now() - timedelta(seconds=seconds)
    else:
        time_of_creation = None


    # Extract country name
    country_name = extract_text(card_tag, '.country_abbr')
    if country_name:
        country_name = country_name.upper()

    # Extract route (source and destination)
    from_location_tag, to_location_tag = card_tag.select('.request_route span')[::2][:2]
    from_location, to_location = parse_route_location(from_location_tag), parse_route_location(to_location_tag)

    # Extract cargo type
    cargo_type = extract_text(card_tag, '.cargo_type')

    # Extract date
    date = extract_text(card_tag, '.date_add')

    # Extract link to the application
    if path := card_tag.select_one('.request_distance').get('href', None):
        link = urljoin(base_url, path)
    else:
        link = None

    # Extract truck type
    truck_type = extract_text(card_tag, '.truck_type')

    # Extract weight
    weight = extract_text(card_tag, '.weight')

    # Extract volume
    volume = extract_text(card_tag, '.cube')

    # Extract distance
    distance = extract_text(card_tag, 'a.distance')

    return SearchCard(time_of_creation, country_name, from_location, to_location, distance, truck_type, weight, volume, cargo_type, date, link)

In [12]:
extract_info(card)

NameError: name 'card' is not defined

In [9]:
with open('../../tests/data/test_page.html') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')

In [13]:
cards = soup.find_all(class_='request_card')
parsed_cards = [extract_info(card) for card in tqdm(cards)]

100%|██████████| 26/26 [00:00<00:00, 182.18it/s]


In [16]:
[c for c in parsed_cards]

[(Creation time: 21 minutes ago
 From: Костанай, KZ, Костанайская  обл.
 To: Челябинск, RU, Челябинская  обл.
 Distance: 642 км
 Date: 16.03
 Truck type: рефрижератор	Weight: 16 т	Volume: 80 м³	Cargo type: тара
 Link: https://www.della.kz/distance/?cities=1237,387,387,1237&rc=23072144341435538),
 (Creation time: 21 minutes ago
 From: Костанай, KZ, Костанайская  обл.
 To: Челябинск, RU, Челябинская  обл.
 Distance: 642 км
 Date: 16.03
 Truck type: рефрижератор	Weight: 16 т	Volume: 80 м³	Cargo type: тара
 Link: https://www.della.kz/distance/?cities=1237,387,387,1237&rc=9223072144341435538),
 (Creation time: 23 minutes ago
 From: Костанай, KZ, Костанайская  обл.
 To: Алашонькоу, CN, 
 Distance: None
 Date: 16.03
 Truck type: тент	Weight: 22 т	Volume: None	Cargo type: семена в биг-бегах
 Link: https://www.della.kz/distance/?cities=1237,780798&rc=9223035180406965991),
 (Creation time: 23 minutes ago
 From: Достык, KZ, Алматинская обл.
 To: Ауденарде, BE, Гент пров.
 Distance: None
 Date: 16